tonite: 

1. vectorstore del dnu con nuevo formato con llama index
2. json ley de bases
3. vectorstore ley de bases
4. linear adapter. 


In [6]:

from llama_index import StorageContext, load_index_from_storage
from llama_index.indices.service_context import ServiceContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import QueryFusionRetriever
from llama_index.node_parser import SimpleNodeParser
from llama_index.embeddings import OpenAIEmbedding
from llama_index.readers import JSONReader
from llama_index.schema import TextNode
from llama_index.llms import OpenAI
import logging
import json
import os

  Using cached pytz-2023.4-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2023.4-py2.py3-none-any.whl (506 kB)


In [7]:
reader = JSONReader()
rawdata_file_path = 'data/LaLeyDeMilei-raw/decreto_flat.json'
data_dict = json.load(open(rawdata_file_path))
documents = reader.load_data(rawdata_file_path)

documents

count = 0
for doc in documents:
    count += 1
    #text = doc.text
    #print(doc.text)

count


#print(documents[0].text)
documents[0].text

#print(documents[0].text)

assert len(documents[0].text.split('\n')) == len(data_dict)

nodes = []
for doc in documents[0].text.split('\n'):
    text = doc
    metadata = '' # 
    node = TextNode(text=text, 
                    metadata=metadata)
    nodes.append(node)

text

'"Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo XVI: REGISTRO AUTOMOTOR (Decreto - Ley N° 6582/58 ratificado por la Ley N° 14.467 (t.o. 1997) y sus modificatorias). Articulo 366": "Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo XVI: REGISTRO AUTOMOTOR (Decreto - Ley N° 6582/58 ratificado por la Ley N° 14.467 (t.o. 1997) y sus modificatorias). Articulo 366: Comuníquese, publíquese, dese a la DIRECCIÓN NACIONAL DEL REGISTRO OFICIAL y archívese."'

In [8]:

nodes = []
for doc in data_dict.values():
    text = doc
    metadata = '' # 
    node = TextNode(text=text, 
                    metadata=metadata)
    nodes.append(node)

nodes

[TextNode(id_='455aa448-deb5-4602-9506-95d14c2de786', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 1: EMERGENCIA. Declárase la emergencia pública en materia económica, financiera, fiscal, administrativa, previsional, tarifaria, sanitaria y social hasta el 31 de diciembre de 2025.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='df93e13c-3cb3-4ee9-8a45-5faf4648f8b7', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 2: DESREGULACIÓN. El Estado Nac

In [11]:
!pip install transformers

  Using cached transformers-4.37.2-py3-none-any.whl.metadata (129 kB)
Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.2 MB/s eta 0:00:0000:0100:01


In [12]:
service_context = ServiceContext.from_defaults(
    embed_model="local:dariolopez/roberta-base-bne-finetuned-msmarco-qa-es-mnrl-mn"
)

/home/tatako/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 764/764 [00:00<00:00, 2.58MB/s]
pytorch_model.bin: 100%|██████████| 499M/499M [00:22<00:00, 22.0MB/s] 
tokenizer_config.json: 100%|██████████| 1.35k/1.35k [00:00<00:00, 7.67MB/s]
vocab.json: 100%|██████████| 851k/851k [00:00<00:00, 1.50MB/s]
merges.txt: 100%|██████████| 509k/509k [00:00<00:00, 3.58MB/s]
tokenizer.json: 100%|██████████| 2.21M/2.21M [00:00<00:00, 3.87MB/s]
special_tokens_map.json: 100%|██████████| 957/957 [00:00<00:00, 3.27MB/s]


In [14]:
!export CUDA_LAUNCH_BLOCKING=1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
vectorstore = VectorStoreIndex(nodes, service_context=service_context)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
counter = 0
for i in data_dict.values():
    counter += 1
    print(f'{counter}: {i}')

1: Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 1: EMERGENCIA. Declárase la emergencia pública en materia económica, financiera, fiscal, administrativa, previsional, tarifaria, sanitaria y social hasta el 31 de diciembre de 2025.
2: Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 2: DESREGULACIÓN. El Estado Nacional promoverá y asegurará la vigencia efectiva, en todo el territorio nacional, de un sistema económico basado en decisiones libres, adoptadas en un ámbito de libre concurrencia, con respeto a la propiedad privada y a los principios constitucionales de libre circulación de bienes, servicios y trabajo.

Para cumplir ese fin, se dispondrá la más amplia desregulación del comercio, los servicios y la industria en todo el territorio nacional y quedarán sin efecto todas las restriccio

In [ ]:
data_dict.values()[1]

TypeError: 'dict_values' object is not subscriptable

In [ ]:
nodes

[TextNode(id_='8017d5ff-6692-4d14-a532-efd8d4e15c95', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='"Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 1": "Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 1: EMERGENCIA. Declárase la emergencia pública en materia económica, financiera, fiscal, administrativa, previsional, tarifaria, sanitaria y social hasta el 31 de diciembre de 2025.",', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='e73a0dad-56ba-4031-b36d-54c3c2d437ee', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='"Decreto de Necesidad y

In [ ]:
TextNode(documents[0].text)

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [29]:
nodes = []
for doc in documents:
    text = doc.text
    metadata = doc.metadata
    node = TextNode(text=text,
                    metadata=metadata)
    nodes.append(node)

nodes[0]

TextNode(id_='968fbdf5-547c-471e-9c25-5720a15f4a44', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='"Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 1": "Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 1: EMERGENCIA. Declárase la emergencia pública en materia económica, financiera, fiscal, administrativa, previsional, tarifaria, sanitaria y social hasta el 31 de diciembre de 2025.",\n"Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 2": "Decreto de Necesidad y Urgencia N° DNU-2023-70-APN-PTE. Fecha 20-12-2023. Titulo I: BASES PARA LA RECONSTRUCCIÓN DE LA ECONOMÍA ARGENTINA. Articulo 2: DESREGULACIÓN. El Estado Nacional prom

In [32]:
dataset = json.load(file_path)

AttributeError: 'str' object has no attribute 'read'